# kiwi로 토큰화 및 로지스틱 회귀 모델 

In [1]:
# 일반방법 : 데이터 불균형 고려안함, N그램 안씀, k폴드교차검증 안함.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from kiwipiepy import Kiwi

# 데이터 불러오기
df = pd.read_csv("../../dataset/merged_labeled_data.csv")

# Kiwi 객체 생성
kiwi = Kiwi()


# 형태소 분석 함수 정의
def tokenize_and_filter(text):
    result = kiwi.analyze(text)[0][0]  # 첫 번째 결과의 토큰 리스트 사용
    tokens = []
    for word, pos, _, _ in result:
        if pos in {"NNG", "NNP", "VV", "VA"}:  # 일반명사, 고유명사, 동사, 형용사
            if pos in {"VV", "VA"}:
                word = word + "다"  # 동사/형용사는 어간 + '다'로 표제어 처리
            tokens.append(word)
    return tokens




In [ ]:
# 토큰화된 텍스트 생성
df["tokens"] = df["text"].astype(str).apply(tokenize_and_filter)
df["joined_tokens"] = df["tokens"].apply(lambda x: " ".join(x))

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["joined_tokens"])
y = df["is_phishing"]

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 간단한 분류 모델 (로지스틱 회귀)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred, output_dict=False)

report

# k 폴드 교차검증 방식, n그램사용, 데이터 불균형 고려

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import numpy as np

# 데이터 다시 로드 (중복 방지)
df = pd.read_csv("../../dataset/merged_labeled_data.csv")

# 형태소 분석기 초기화
kiwi = Kiwi()


# 형태소 분석 함수 (명사/동사/형용사/고유명사, 표제어 처리 포함)
def tokenize_and_filter(text):
    result = kiwi.analyze(text)[0][0]
    tokens = []
    for word, pos, _, _ in result:
        if pos in {"NNG", "NNP", "VV", "VA"}:
            if pos in {"VV", "VA"}:
                word = word + "다"
            tokens.append(word)
    return " ".join(tokens)


# 텍스트 전처리
df["processed_text"] = df["text"].astype(str).apply(tokenize_and_filter)

X = df["processed_text"]
y = df["is_phishing"]

# 파이프라인 구성: TF-IDF + 로지스틱 회귀 (클래스 불균형 고려, n-gram 적용)
pipeline = Pipeline(
    [
        (
            "tfidf",
            TfidfVectorizer(
                sublinear_tf=True,
                min_df=5,
                max_df=0.9,
                ngram_range=(1, 2),  # uni-gram + bi-gram
            ),
        ),
        ("clf", LogisticRegression(max_iter=1000, class_weight="balanced")),
    ]
)

# Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 예측
y_pred = cross_val_predict(pipeline, X, y, cv=skf)

# 평가 리포트 출력
report = classification_report(y, y_pred)
report

# 위의 두 방식을 종합하여 비교까지 진행하는 코드

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import numpy as np
from kiwipiepy import Kiwi

# 샘플 데이터 로드 경로 수정 필요 시 여기를 변경하세요
data_path = "../../dataset/merged_labeled_data.csv"
df = pd.read_csv(data_path)

# Kiwi 형태소 분석기
kiwi = Kiwi()


# 토큰화 및 표제어 처리
def tokenize_and_filter(text):
    result = kiwi.analyze(text)[0][0]
    tokens = []
    for word, pos, _, _ in result:
        if pos in {"NNG", "NNP", "VV", "VA"}:
            if pos in {"VV", "VA"}:
                word = word + "다"
            tokens.append(word)
    return " ".join(tokens)


# 텍스트 전처리
df["processed_text"] = df["text"].astype(str).apply(tokenize_and_filter)

X = df["processed_text"]
y = df["is_phishing"]

# ▶ 방식 1: 단순 train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

pipeline_simple = Pipeline(
    [
        ("tfidf", TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.9)),
        ("clf", LogisticRegression(max_iter=1000, class_weight="balanced")),
    ]
)
pipeline_simple.fit(X_train, y_train)
y_pred_simple = pipeline_simple.predict(X_test)
report_simple = classification_report(y_test, y_pred_simple, output_dict=True)

# ▶ 방식 2: K-Fold 교차 검증 + ngram 적용
pipeline_kfold = Pipeline(
    [
        (
            "tfidf",
            TfidfVectorizer(
                sublinear_tf=True, min_df=5, max_df=0.9, ngram_range=(1, 2)
            ),
        ),
        ("clf", LogisticRegression(max_iter=1000, class_weight="balanced")),
    ]
)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_kfold = cross_val_predict(pipeline_kfold, X, y, cv=skf)
report_kfold = classification_report(y, y_pred_kfold, output_dict=True)

# 결과 비교를 위해 정리
comparison = {"Metric": [], "Simple Split": [], "K-Fold": []}

for label in ["0", "1", "accuracy", "macro avg", "weighted avg"]:
    if label == "accuracy":
        comparison["Metric"].append("Accuracy")
        comparison["Simple Split"].append(report_simple["accuracy"])
        comparison["K-Fold"].append(report_kfold["accuracy"])
    else:
        for metric in ["precision", "recall", "f1-score"]:
            metric_name = (
                f"{label} {metric}" if label in ["0", "1"] else f"{label} {metric}"
            )
            comparison["Metric"].append(metric_name)
            comparison["Simple Split"].append(report_simple[label][metric])
            comparison["K-Fold"].append(report_kfold[label][metric])



In [3]:
import pandas as pd
from IPython.display import display

result_df = pd.DataFrame(comparison)
display(result_df)

# 콘솔에서 직접 출력
print(result_df)

# 또는 CSV 파일로 저장
result_df.to_csv("../../dataset/model_comparison_result.csv", index=False)

,Metric,Simple Split,K-Fold
0,0 precision,0.999161,0.996500
1,0 recall,0.992500,0.996500
2,0 f1-score,0.995819,0.996500
3,1 precision,0.978469,0.989741
4,1 recall,0.997561,0.989741
5,1 f1-score,0.987923,0.989741
6,Accuracy,0.993789,0.994781
7,macro avg precision,0.988815,0.993121
8,macro avg recall,0.995030,0.993121
9,macro avg f1-score,0.991871,0.993121


                    Metric  Simple Split    K-Fold
0              0 precision      0.999161  0.996500
1                 0 recall      0.992500  0.996500
2               0 f1-score      0.995819  0.996500
3              1 precision      0.978469  0.989741
4                 1 recall      0.997561  0.989741
5               1 f1-score      0.987923  0.989741
6                 Accuracy      0.993789  0.994781
7      macro avg precision      0.988815  0.993121
8         macro avg recall      0.995030  0.993121
9       macro avg f1-score      0.991871  0.993121
10  weighted avg precision      0.993892  0.994781
11     weighted avg recall      0.993789  0.994781
12   weighted avg f1-score      0.993808  0.994781


In [5]:
# 모델 저장
import joblib
import os

# 모델 저장 디렉토리 만들기
os.makedirs("../../models", exist_ok=True)

# 1. Simple Split 모델 저장
joblib.dump(pipeline_simple, "../../models/pipeline_simple.pkl")

# 2. K-Fold 모델 → 전체 데이터로 다시 학습 후 저장
pipeline_kfold.fit(X, y)
joblib.dump(pipeline_kfold, "../../models/pipeline_kfold.pkl")

['../../models/pipeline_kfold.pkl']

# 피싱 논피싱 단순 판별

In [ ]:
# 저장된 모델 불러오기
loaded_model = joblib.load("../../models/1차모델_원본데이터_pipeline_Ngram_kfold.pkl")
loaded_model.predict(["수사기관을 사칭하고 계좌를 요구하는 대화입니다"])  # 판별할 문장을 넣는곳

array([1])

# 퍼센테이지로 구간 별로 판별함 

In [8]:
# 모델 불러오기
import joblib

loaded_model = joblib.load("../../models/1차모델_원본데이터_pipeline_Ngram_kfold.pkl")

In [9]:
def classify_with_hold(prob, low=0.3, high=0.7):
    if prob < low:
        return 0  # 일반 대화
    elif prob > high:
        return 1  # 보이스피싱
    else:
        return -1  # 보류 → 2차 모델로

In [ ]:
# 예측 대상 문장
input_text = [
    "그러시면 71년생 남성 사람은 아십니까? 지금 강성호 씨는 돈을 주고 샀다고 짓을 하는데 본인께서 감상으로 저녁 먹읍시다 말씀이세요. 알다시피 금융거래실명법 원칙에 따라 통장을 개설하면 본인이 직접 신분증을 지참하고 은행에 가서 하지 마 통장 발급 되지 않습니까네? 본인이 모른다고 해서 5만 원은 살 거는 아니고요. 보이 입장도 이해를 하고 환경을 수 없지만요. 이번 사건은 방송을 하는 사람이 불법도박사이트로 운영하면서 대포통장이 필요하니까 50원을 포함한 전국의 132명의 자들에게 통장을 구매했다고 짓을 해서 연락을 드리면 부분이고요."
]  

# 보이스피싱(클래스 1) 확률만 추출
proba = loaded_model.predict_proba(input_text)[0][1]

# 보류 구간 판별 적용
final_result = classify_with_hold(proba)

print(f"예측 확률: {proba:.4f}")
print(f"1차 판별 결과: {final_result} (0: 일반, 1: 피싱, -1: 보류)")

예측 확률: 0.6084
1차 판별 결과: -1 (0: 일반, 1: 피싱, -1: 보류)


# 앙상블 Bagging 1차 모델 학습


In [1]:
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib
import os

# 데이터 로드
data_path = "../../dataset/merged_labeled_data.csv"
df = pd.read_csv(data_path)

# Kiwi 토큰화 + 표제어 처리
kiwi = Kiwi()


def tokenize_and_filter(text):
    result = kiwi.analyze(text)[0][0]
    tokens = []
    for word, pos, _, _ in result:
        if pos in {"NNG", "NNP", "VV", "VA"}:
            if pos in {"VV", "VA"}:
                word = word + "다"
            tokens.append(word)
    return " ".join(tokens)


df["processed_text"] = df["text"].astype(str).apply(tokenize_and_filter)

X = df["processed_text"]
y = df["is_phishing"]

# ▶ 앙상블 모델 파이프라인 (TF-IDF + RandomForest)
pipeline_rf = Pipeline(
    [
        (
            "tfidf",
            TfidfVectorizer(
                sublinear_tf=True, min_df=5, max_df=0.9, ngram_range=(1, 2)
            ),
        ),
        (
            "clf",
            RandomForestClassifier(
                n_estimators=100, random_state=42, class_weight="balanced"
            ),
        ),
    ]
)

# ▶ K-Fold 교차 검증 성능 측정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_rf = cross_val_predict(pipeline_rf, X, y, cv=skf)

# ▶ 성능 리포트 출력
report_rf = classification_report(y, y_pred_rf, output_dict=False)
print(report_rf)

# ▶ 전체 데이터로 다시 학습 후 저장
pipeline_rf.fit(X, y)

# 저장 경로
os.makedirs("../../models", exist_ok=True)
joblib.dump(pipeline_rf, "../../models/1차모델_원본데이터_pipeline_rf.pkl")

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6000
           1       1.00      0.96      0.98      2047

    accuracy                           0.99      8047
   macro avg       0.99      0.98      0.99      8047
weighted avg       0.99      0.99      0.99      8047



['../../models/1차모델_원본데이터_pipeline_rf.pkl']

# 앙상블 Stacking 1차 모델 

In [ ]:
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.ensemble import (
    StackingClassifier,
    RandomForestClassifier,
    GradientBoostingClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib
import os

# 1. 데이터 불러오기
df = pd.read_csv("../../dataset/merged_labeled_data.csv")

# 2. 형태소 분석기 설정
kiwi = Kiwi()


def tokenize_and_filter(text):
    result = kiwi.analyze(text)[0][0]
    tokens = []
    for word, pos, _, _ in result:
        if pos in {"NNG", "NNP", "VV", "VA"}:
            if pos in {"VV", "VA"}:
                word += "다"
            tokens.append(word)
    return " ".join(tokens)


# 3. 전처리
df["processed_text"] = df["text"].astype(str).apply(tokenize_and_filter)
X = df["processed_text"]
y = df["is_phishing"]

# 4. 스태킹 모델 구성
base_models = [
    ("lr", LogisticRegression(max_iter=1000, class_weight="balanced")),
    (
        "rf",
        RandomForestClassifier(
            n_estimators=100, class_weight="balanced", random_state=42
        ),
    ),
    ("gb", GradientBoostingClassifier(n_estimators=100, random_state=42)),
]

meta_model = LogisticRegression(max_iter=1000)

stacked_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,  # 내부에서 K-Fold
    n_jobs=-1,
    passthrough=False,  # True면 base 모델의 입력도 같이 전달됨
)

# 5. 전체 파이프라인 (TF-IDF → 스태킹 모델)
pipeline_stacking = Pipeline(
    [
        (
            "tfidf",
            TfidfVectorizer(
                sublinear_tf=True, min_df=5, max_df=0.9, ngram_range=(1, 2)
            ),
        ),
        ("clf", stacked_clf),
    ]
)

# 6. K-Fold 기반 평가
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_stack = cross_val_predict(pipeline_stacking, X, y, cv=skf)

# 7. 성능 출력
report = classification_report(y, y_pred_stack)
print(report)

# 8. 최종 전체 학습 및 저장
pipeline_stacking.fit(X, y)
os.makedirs("../../models", exist_ok=True)
joblib.dump(pipeline_stacking, "../../models/pipeline_stacking.pkl")